<a href="https://colab.research.google.com/github/coelhu/CI2/blob/Aula_1/atividade06_leitor_de_captcha.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Atividade 06
# Aluno: André Coelho Ramos


import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pathlib import Path
from collections import Counter

from sklearn.model_selection import train_test_split
from sympy import true

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

seed = 1234
np.random.seed(seed)
tf.random.set_seed(seed)

# Images captcha salvos localmente na pasta captcha_images_v2/ dentro da pasta do projeto
data_dir = Path("../input/captcha-version-2-images/samples/")

# Pega uma lista de todas as imagens
images = list(data_dir.glob("*.png"))
print("Número de imagens encontradas: ", len(images))



Número de imagens encontradas:  1040


In [ ]:
# Salva todos os caracteres dos nomes dos captchas em um array
characters = set()

# Array para salvar o tamanho de cada captcha
captcha_length = []

# Armazena informações da imagem e seu label ou nome
dataset = []

# Armazena as informações de todas as imagens da pasta
for img_path in images:
    # 1. Pega o nome associado com cada imagem
    label = img_path.name.split(".png")[0]
    # 2. Salva o tamanho
    captcha_length.append(len(label))
    # 3. Salva a informação de imagem e label
    dataset.append((str(img_path), label))
    # 4. Salva os caracteres presents no label
    for ch in label:
        characters.add(ch)

# Ordena os caracteres
characters = sorted(characters)

# Converte em dataframe o dataset
dataset = pd.DataFrame(dataset, columns=["img_path", "label"], index=None)

# Mistura o dataset
dataset = dataset.sample(frac=1.0).reset_index(drop=True)

# Divide o dataset em dados de treinamento e dados de validação, com 10% para o teste
training_data, validation_data = train_test_split(
    dataset, test_size=0.1, random_state=seed
)
training_data = training_data.reset_index(drop=True)
validation_data = validation_data.reset_index(drop=True)

# Mapeia labels para números
char_to_labels = {char: idx for idx, char in enumerate(characters)}

# Mapeia labels numericos para texto
labels_to_char = {val: key for key, val in char_to_labels.items()}


# Função para verificar se existem imagens corrompidas
def is_valid_captcha(captcha):
    for ch in captcha:
        if not ch in characters:
            return False
    return True



#A função gera matrizes de imagens e rótulos a partir de um dataframe do pandas.
#A função primeiro cria arrays vazios para imagens e rótulos.
# Em seguida, ele percorre o dataframe e lê cada imagem.
# As imagens são convertidas para tons de cinza e redimensionadas, se necessário.
# As imagens são então normalizadas para o intervalo [0, 1].
# Os rótulos também são convertidos em uma matriz numpy.
def generate_arrays(df, resize=True, img_height=50, img_width=200):
    """Gera arrays de imagem e labels do dataframe

    Args:
        df: dataframe que queremos ler
        resize (bool)    :  redimencionar as imagens ou não
        img_weidth (int): largura da imagem redimencionada
        img_height (int): altura da imagem redimencionada

    Retorna:
        images (ndarray): imagens em escala de cinza
        labels (ndarray): labels correspondentes
    """

    num_items = len(df)
    images = np.zeros((num_items, img_height, img_width), dtype=np.float32)
    labels = [0] * num_items

    for i in range(num_items):
        img = cv2.imread(df["img_path"][i])
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        if resize:
            img = cv2.resize(img, (img_width, img_height))

        img = (img / 255.0).astype(np.float32)
        label = df["label"][i]

        # Somente adiciona se é um captcha válido
        if is_valid_captcha(label):
            images[i, :, :] = img
            labels[i] = label

    return images, np.array(labels)


# Constroi dados de treinamento
training_data, training_labels = generate_arrays(df=training_data)

# Constroi dados de validação
validation_data, validation_labels = generate_arrays(df=validation_data)


# A função DataGenerator é uma classe que gera lotes de dados para um modelo de reconhecimento captcha.
# Para cada imagem do lote, o método:
#   Transpõe a imagem.
#   Adiciona uma dimensão extra à imagem.
#   Obtém o rótulo correspondente.
#   Inclui o par somente se o captcha for válido.
#    Retorna:
#        batch_inputs: um dicionario contendo entradas de batch
#        batch_labels: um batch de labels correspondentes
class DataGenerator(keras.utils.Sequence):
    def __init__(
        self,
        data,
        labels,
        char_map,
        batch_size=16,
        img_width=200,
        img_height=50,
        downsample_factor=4,
        max_length=5,
        shuffle=True,
    ):
        self.data = data
        self.labels = labels
        self.char_map = char_map
        self.batch_size = batch_size
        self.img_width = img_width
        self.img_height = img_height
        self.downsample_factor = downsample_factor
        self.max_length = max_length
        self.shuffle = shuffle
        self.indices = np.arange(len(data))
        self.on_epoch_end()

    def __len__(self):
        return int(np.ceil(len(self.data) / self.batch_size))

    def __getitem__(self, idx):
        # 1. Pega o próximo batch de indices
        curr_batch_idx = self.indices[
            idx * self.batch_size : (idx + 1) * self.batch_size
        ]

        # 2. Isso não é necessário mas pode ajudar a alvar memória
        # porque nem todos os batches devem ter elementos iguais ao de tamanho do batch
        batch_len = len(curr_batch_idx)

        # 3. Instancia os arrays de batchs
        batch_images = np.ones(
            (batch_len, self.img_width, self.img_height, 1), dtype=np.float32
        )
        batch_labels = np.ones((batch_len, self.max_length), dtype=np.float32)
        input_length = np.ones((batch_len, 1), dtype=np.int64) * (
            self.img_width // self.downsample_factor - 2
        )
        label_length = np.zeros((batch_len, 1), dtype=np.int64)

        for j, idx in enumerate(curr_batch_idx):
            # 1. Transposta a imagem
            img = self.data[idx].T
            # 2. Adiciona dimenção extra
            img = np.expand_dims(img, axis=-1)
            # 3. Pega o label correspondente
            text = self.labels[idx]
            # 4. Inclui o par somente se o catpcha é válido
            if is_valid_captcha(text):
                label = [self.char_map[ch] for ch in text]
                batch_images[j] = img
                batch_labels[j] = label
                label_length[j] = len(text)

        batch_inputs = {
            "input_data": batch_images,
            "input_label": batch_labels,
            "input_length": input_length,
            "label_length": label_length,
        }
        return batch_inputs, np.zeros(batch_len).astype(np.float32)

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indices)


# Tamanh do batch para treino e validação
batch_size = 16

# Tamanho desejado das imagens
img_width = 200
img_height = 50

# Fator de downsampling da imagem por blocos convolucionais
downsample_factor = 4

# Tamanho máximo de qualquer captcha nos dados
max_length = 5

# Objeto gerador para os dados de treinamento
train_data_generator = DataGenerator(
    data=training_data,
    labels=training_labels,
    char_map=char_to_labels,
    batch_size=batch_size,
    img_width=img_width,
    img_height=img_height,
    downsample_factor=downsample_factor,
    max_length=max_length,
    shuffle=True,
)

# Objeto gerador para os dados de validação
valid_data_generator = DataGenerator(
    data=validation_data,
    labels=validation_labels,
    char_map=char_to_labels,
    batch_size=batch_size,
    img_width=img_width,
    img_height=img_height,
    downsample_factor=downsample_factor,
    max_length=max_length,
    shuffle=False,
)


# A função CTCLayer é uma camada Keras personalizada que calcula a perda de CTC para um modelo de
# reconhecimento de captcha.
class CTCLayer(layers.Layer):
    def __init__(self, name=None):
        super().__init__(name=name)
        self.loss_fn = keras.backend.ctc_batch_cost

    def call(self, y_true, y_pred, input_length, label_length):
        # Calcula a perda do training-time e adicionar ao layer
        loss = self.loss_fn(y_true, y_pred, input_length, label_length)
        self.add_loss(loss)

        # Durante o teste retorna somente a perda
        return loss


# A função build_model() constrói um modelo de reconhecimento captcha baseado em rede neural convolucional (CNN).
# A função primeiro define as camadas de entrada para as imagens, rótulos, comprimentos de entrada e
# comprimentos de rótulos.
# Em seguida, define blocos da CNN
# A função então define blocos RNN
def build_model():
    # Adiciona ao modelo
    input_img = layers.Input(
        shape=(img_width, img_height, 1), name="input_data", dtype="float32"
    )
    labels = layers.Input(name="input_label", shape=[max_length], dtype="float32")
    input_length = layers.Input(name="input_length", shape=[1], dtype="int64")
    label_length = layers.Input(name="label_length", shape=[1], dtype="int64")

    # Primeiro block convulacional
    x = layers.Conv2D(
        32,
        (3, 3),
        activation="relu",
        kernel_initializer="he_normal",
        padding="same",
        name="Conv1",
    )(input_img)
    x = layers.MaxPooling2D((2, 2), name="pool1")(x)

    # Segundo block convulacional
    x = layers.Conv2D(
        64,
        (3, 3),
        activation="relu",
        kernel_initializer="he_normal",
        padding="same",
        name="Conv2",
    )(x)
    x = layers.MaxPooling2D((2, 2), name="pool2")(x)

    new_shape = ((img_width // 4), (img_height // 4) * 64)
    x = layers.Reshape(target_shape=new_shape, name="reshape")(x)
    x = layers.Dense(64, activation="relu", name="dense1")(x)
    x = layers.Dropout(0.2)(x)

    # RNNs
    x = layers.Bidirectional(layers.LSTM(128, return_sequences=True, dropout=0.2))(x)
    x = layers.Bidirectional(layers.LSTM(64, return_sequences=True, dropout=0.25))(x)

    # Precição
    x = layers.Dense(
        len(characters) + 1,
        activation="softmax",
        name="dense2",
        kernel_initializer="he_normal",
    )(x)

    # Calcula CTC
    output = CTCLayer(name="ctc_loss")(labels, x, input_length, label_length)

    # Define o modelo
    model = keras.models.Model(
        inputs=[input_img, labels, input_length, label_length],
        outputs=output,
        name="ocr_model_v1",
    )

    # Otimizar
    sgd = keras.optimizers.SGD(
        learning_rate=0.002, momentum=0.9, nesterov=True, clipnorm=5
    )

    # Compila o model o retorna
    model.compile(optimizer=sgd)
    return model


# Construir e imprimir um resumo do modelo de reconhecimento de captcha.
model = build_model()
model.summary()

# Define um retorno de chamada EarlyStopping. Esse retorno de chamada interrompe o treinamento do modelo
# antecipadamente se a perda de validação não melhorar por um determinado número de épocas (paciência).
# Os melhores pesos do modelo são restaurados quando o callback interrompe o treinamento.
es = keras.callbacks.EarlyStopping(
    monitor="val_loss", patience=5, restore_best_weights=True
)


# A função treina o modelo usando o train_data_generator e avalia o modelo usando o valid_data_generator.
# O treinamento é feito para épocas e o retorno de chamada es é usado para interromper o
# treinamento antecipadamente se a perda de validação não melhorar.
history = model.fit(
    train_data_generator,
    validation_data=valid_data_generator,
    epochs=50,
    callbacks=[es],
)

# A função cria um novo modelo que é um subconjunto do modelo original.
# O novo modelo inclui apenas a camada de entrada e a camada densa com 46 neurônios.
# Isso é chamado de modelo ajustado.
prediction_model = keras.models.Model(
    model.get_layer(name="input_data").input, model.get_layer(name="dense2").output
)
prediction_model.summary()


# A função decodifica as previsões do modelo de reconhecimento captcha.
# A função primeiro remove as duas últimas colunas das previsões, que não são usadas para decodificação.
# Em seguida, ele calcula os comprimentos de entrada, que são iguais para todas as previsões.
# A função então usa o backend Keras para decodificar as previsões usando o algoritmo de pesquisa ganancioso.
# O algoritmo de busca ganancioso simplesmente escolhe o rótulo mais provável em cada intervalo de tempo.
def decode_batch_predictions(pred):
    pred = pred[:, :-2]
    input_len = np.ones(pred.shape[0]) * pred.shape[1]

    # Use busca do tipo greedy search
    results = keras.backend.ctc_decode(pred, input_length=input_len, greedy=True)[0][0]

    # Loop pelos resultados e retorna o texto
    output_text = []
    for res in results.numpy():
        outstr = ""
        for c in res:
            if c < len(characters) and c >= 0:
                outstr += labels_to_char[c]
        output_text.append(outstr)

    # Retorno os resultados finais
    return output_text

Model: "ocr_model_v1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_data (InputLayer)        [(None, 200, 50, 1)  0           []                               
                                ]                                                                 
                                                                                                  
 Conv1 (Conv2D)                 (None, 200, 50, 32)  320         ['input_data[0][0]']             
                                                                                                  
 pool1 (MaxPooling2D)           (None, 100, 25, 32)  0           ['Conv1[0][0]']                  
                                                                                                  
 Conv2 (Conv2D)                 (None, 100, 25, 64)  18496       ['pool1[0][0]']       

TESTA SOMENTE UMA IMAGEM> 8n5p3.png

USANDO CÓDIGO PRÉVIO, PORÉM ADAPTADO ELE PARA SOMENTE UMA IMAGEM

In [ ]:
images_single = list(data_dir.glob("8n5p3.png"))
print("Number of images found: ", len(images_single))

characters_simples = set()
captcha_length_simples = []
dataset_simples = []
for img_path in images_single:
    label = img_path.name.split(".png")[0]
    captcha_length_simples.append(len(label))
    dataset_simples.append((str(img_path), label))
    for ch in label:
        characters_simples.add(ch)

characters_simples = sorted(characters_simples)
dataset_simples = pd.DataFrame(
    dataset_simples, columns=["img_path", "label"], index=None
)

validation_data_simples, validation_labels_simples = generate_arrays(df=dataset_simples)

# Gera um objeto gerador para essa única imagem
imagen_teste = DataGenerator(
    data=validation_data_simples,
    labels=validation_labels_simples,
    char_map=char_to_labels,
    batch_size=1,
    img_width=img_width,
    img_height=img_height,
    downsample_factor=downsample_factor,
    max_length=max_length,
    shuffle=False,
)

# executa somente uma interação para pegar essa imagem e analisar ela
for p, (inp_value, _) in enumerate(imagen_teste):
    bs = inp_value["input_data"].shape[0]
    X_data = inp_value["input_data"]
    labels = inp_value["input_label"]

    preds = prediction_model.predict(X_data)
    pred_texts = decode_batch_predictions(preds)

    orig_texts = []
    for label in labels:
        text = "".join([labels_to_char[int(x)] for x in label])
        orig_texts.append(text)

    for i in range(bs):
        print(f"Ground truth: {orig_texts[i]} \t Predicted: {pred_texts[i]}")
    break

####################################################
## RESULTADO
# O código consegue reconhecer a imagem 8n5p3.png
####################################################

# Ground truth: 8n5p3      Predicted: 8n5p3

Number of images found:  1
1/1 [==============================] - 2s 2s/step
Ground truth: 8n5p3 	 Predicted: 8n5p3
